<a href="https://colab.research.google.com/github/Suma-Marri/COP6526_Homework1/blob/main/COP6526_Homework1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
import numpy as np
import re, string
from pyspark import SparkContext
from pyspark import SparkConf
import itertools
from pyspark.sql import Row
from collections import OrderedDict

In [3]:
sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
infile= sc.textFile("/content/drive/MyDrive/input.txt")

In [6]:
infile.take(5)

['The Project Gutenberg EBook of The Adventures of Sherlock Holmes',
 'by Sir Arthur Conan Doyle',
 '(#15 in our series by Sir Arthur Conan Doyle)',
 '',
 'Copyright laws are changing all over the world. Be sure to check the']

In [7]:
def clean_data(x):
    cleanfile=x.lower()
    cleanfile=re.sub("[^a-z ]","",cleanfile)
    cleanfile=re.sub("[\s]+"," ",cleanfile)
    cleanfile=re.sub("^[\s]+","",cleanfile)
    cleanfile=re.sub("[\s]+$","",cleanfile)
    cleanfile=re.sub(r"!|\'|\"|;|:",'', cleanfile)
    cleanfile=re.sub(r'\d','', cleanfile)
    return cleanfile

In [8]:
def getlist(rawdata):
    dictionary = {}
    for x in range(len(rawdata)):
        for y in range(len(rawdata)):
            if x!=y:
                key1= rawdata[x]+","+rawdata[y]
                if (key1 not in dictionary):
                    dictionary[key1] = 1
                else:
                    if (rawdata[x] not in rawdata[0:x]):
                        dictionary[key1]=dictionary[key1]+1
    return dictionary.items()

In [10]:
clean_rdd =infile.map(lambda x: clean_data(x).split(" "))

In [11]:
clean_rdd =clean_rdd.map(lambda line: getlist(line))

In [12]:
clean_rdd= clean_rdd.flatMap(lambda x:x)

In [13]:
clean_rdd= clean_rdd.map(lambda x: (x[0], x[1]))

In [14]:
clean_rdd= clean_rdd.reduceByKey(lambda x,y: x+y)

In [15]:
clean_rdd.collect()

[('the,gutenberg', 4),
 ('the,the', 35),
 ('project,gutenberg', 5),
 ('project,adventures', 2),
 ('project,holmes', 2),
 ('gutenberg,project', 5),
 ('gutenberg,ebook', 4),
 ('gutenberg,of', 5),
 ('gutenberg,holmes', 2),
 ('ebook,the', 5),
 ('ebook,gutenberg', 4),
 ('ebook,adventures', 2),
 ('ebook,sherlock', 2),
 ('ebook,holmes', 2),
 ('of,the', 74),
 ('of,gutenberg', 3),
 ('adventures,of', 6),
 ('sherlock,adventures', 4),
 ('sherlock,holmes', 5),
 ('holmes,project', 2),
 ('holmes,of', 11),
 ('holmes,adventures', 4),
 ('by,doyle', 3),
 ('sir,doyle', 4),
 ('arthur,by', 3),
 ('arthur,conan', 4),
 ('arthur,doyle', 4),
 ('conan,doyle', 4),
 ('doyle,sir', 4),
 ('in,series', 1),
 ('in,by', 1),
 ('in,doyle', 1),
 ('our,series', 1),
 ('our,by', 1),
 ('our,arthur', 1),
 ('our,doyle', 1),
 ('series,in', 1),
 ('series,our', 1),
 ('series,by', 1),
 ('series,arthur', 1),
 ('series,doyle', 1),
 ('by,series', 1),
 ('sir,in', 1),
 ('sir,our', 1),
 ('sir,series', 1),
 ('arthur,in', 1),
 ('arthur,our', 